In [1]:
import numpy as np
from ipalm import *
import matplotlib.pyplot as plt
import math

In [2]:
##### solve sparse PCA as test

### generate toy matrix to factorize

M=33 #dimensions
N=25 #dimensions
r=6 #rank of matrix


#subscript of 't' denotes true parameter used to generate toy data

p_sparse=0.25

#let's make these things sparse

U_t=np.random.normal(size=[M, r])
V_t=np.random.normal(size=[r, N])

sp_mask_inds=[np.random.choice( np.array([i for i in range(r)]),\
                               int(r*p_sparse)+1 )\
              for x in range(M)]

sp_mask_t=np.array([[1 if i in x else 0 for i in range(r)] for x in sp_mask_inds])


U_t=np.multiply(U_t, sp_mask_t)

U_t=np.divide(U_t, np.linalg.norm(U_t,axis=0))
V_t=np.divide(V_t, np.expand_dims(np.linalg.norm(V_t,axis=1),axis=1))

s_t=np.random.uniform(size=[r,])
s_t=sorted(s_t)

S_t=np.diag(s_t)

P_t=np.dot(np.dot(U_t,S_t),V_t)

plt.figure()
plt.subplot(1,3,1)
plt.imshow(U_t, interpolation='none')
plt.colorbar()
plt.subplot(1,3,2)
plt.imshow(S_t, interpolation='none')
plt.colorbar()
plt.subplot(1,3,3)
plt.imshow(V_t, interpolation='none')
plt.colorbar()

plt.figure()
plt.imshow(P_t, interpolation='none')
plt.colorbar()

plt.show()


/usr/lib/pymodules/python2.7/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


In [5]:
### the iPALM algorithm
### no backtracking

### this specific problem
# solve 1/2*|| Y-Xa*Xb ||_F^2 + lam*||xa||_1 + lam/2*||xb||_2^2


### inputs

#inits

#functions
grad_a=lambda xs, opts: \
    np.dot(np.dot(xs[0].reshape([opts['M'], opts['r']]),\
                  xs[1].reshape([opts['r'], opts['N']]))-opts['Y'],\
           xs[1].reshape([opts['r'], opts['N']]).T).\
    reshape([opts['M']*opts['r'],])

grad_b=lambda xs, opts: \
    np.dot(xs[0].reshape([opts['M'], opts['r']]).T,\
           np.dot(np.reshape(xs[0], [opts['M'], opts['r']]),\
                  np.reshape(xs[1], [opts['r'], opts['N']]))-opts['Y']).\
    reshape([opts['r']*opts['N'],])

prox_a=lambda v, t, opts: \
    np.sign(v)*np.maximum(0.0, np.abs(v)-opts['lam']/t)

prox_b=lambda v, t, opts: \
    np.divide(v, (1.0+opts['lam']/t))

#optimization params
nit=1e3 #max iterations before termination
tol=1e-4 #error threshold for termination

opts={}
opts['lam']=0.00001;
opts['r']=r
opts['M']=M
opts['N']=N
opts['Y']=P_t

opts['nit']=nit
opts['tol']=tol

grads=[grad_a, grad_b]
proxs=[prox_a, prox_b]

lip_a = lambda xs, opts: np.power(np.linalg.norm(xs[1]), 2)
lip_b = lambda xs, opts: np.power(np.linalg.norm(xs[0]), 2)

lips=[lip_a, lip_b]
opts['lips']=lips
xs2 = ipalm(grads, proxs, [M*r, r*N], opts)
#print(xs2)
[xa2, xb2] = xs2
print(opts)

P_h=np.dot(xs2[0].reshape([M,r]),\
           xs2[1].reshape([r,N]))

{'lam': 1e-05, 'M': 33, 'N': 25, 'lips': [<function <lambda> at 0x7f13f52a6cf8>, <function <lambda> at 0x7f13f52a6de8>], 'r': 6, 'tol': 0.0001, 'Y': array([[ -7.22150534e-03,  -4.44401117e-02,  -5.57685252e-02,
          2.71912318e-02,  -1.44964233e-02,   2.93291469e-02,
          1.52005890e-02,   6.89743645e-02,  -1.68755497e-02,
          2.74821679e-02,   4.09503149e-02,   1.89003119e-02,
          6.30642603e-02,   2.88521842e-02,  -3.59443250e-02,
         -4.79741294e-02,  -1.45904248e-02,   2.17838125e-02,
         -3.11517982e-02,  -8.36387737e-03,  -3.62585137e-02,
          1.66079591e-02,  -1.78310243e-02,  -9.28109179e-03,
          5.90349303e-02],
       [ -2.84364240e-01,  -8.71773416e-02,   3.98869852e-01,
          6.08340171e-02,  -1.40994414e-01,   2.92210478e-02,
          1.20617874e-01,   1.76351948e-01,  -4.00006700e-01,
         -1.22218682e-01,  -6.29595421e-02,  -1.41435661e-02,
          3.73470723e-02,  -2.20247484e-01,  -1.00982226e-01,
          2.100779

In [4]:
cmi=np.min([P_t,P_h])
cma=np.max([P_t,P_h])

plt.figure()
plt.imshow(P_h,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.figure()
plt.imshow(P_t,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.figure()
plt.imshow(P_t-P_h,interpolation='none')
plt.clim(cmi,cma)
plt.colorbar()

plt.show()
